In [147]:
import requests
from bs4 import BeautifulSoup
import re
import gspread
import pandas as pd
from collections import Counter
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe

In [148]:
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1KrxeJ5sPvirGUhL9i0E4QmwiMQg5O8odtTOBXdEGuLg/edit"

In [149]:
def authenticate_gspread():
    # Define the scope
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    
    # Provide the path to your Google Service Account JSON key file
    credentials = ServiceAccountCredentials.from_json_keyfile_name({'YOUR KEY HERE'}, scope)
    
    # Authenticate and initialize the client
    client = gspread.authorize(credentials)
    
    return client

In [150]:
# Function to upload the dataframe to a specific tab in the spreadsheet
def upload_to_google_sheets(spreadsheet_url, sheet_name, dataframe):
    # Authenticate the client
    client = authenticate_gspread()
    
    # Open the spreadsheet by URL
    spreadsheet = client.open_by_url(spreadsheet_url)
    
    # Select the specific worksheet (tab) by name
    worksheet = spreadsheet.worksheet(sheet_name)
    
    # Clear the existing content in the worksheet
    worksheet.clear()
    
    # Write the dataframe to the worksheet
    set_with_dataframe(worksheet, dataframe, include_index = False, include_column_header=True, resize=False)

In [151]:
# URL of the page to scrape
URL = "https://www.federalregister.gov/presidential-documents/executive-orders/donald-trump/2025"

# Define headers to mimic a real browser request
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

# Send a GET request to fetch the HTML content with headers
response = requests.get(URL, headers=HEADERS)

In [152]:
SOUP = BeautifulSoup(response.text, "html.parser")

# Find the container with data on one EO
CONTENT = SOUP.find_all(class_="presidential-document-wrapper")

# Create an empty list to hold the data
data = []

# Loop through each container and extract relevant information
for document in CONTENT:
    eo_number = document.find("h5").text.strip().replace(":","").replace("EO ","") if document.find("h5") else None
    url_base ='https://www.federalregister.gov'
    url_unique = document.find("a")["href"] if document.find("a") else None
    url = url_base+url_unique
    title = document.find("a").text if document.find("a") else None
    
    metadata = document.find("div", {'class':'metadata_wrapper'})
    date_signed = metadata.find_all("dd")[0].text
    date_published = metadata.find_all("dd")[1].text.strip()
    fr_citation = metadata.find_all("dd")[2].text
    fr_doc_number = metadata.find_all("dd")[3].text.strip()
    number_of_pages = metadata.find_all("dd")[4].text.strip()
    link_to_pdf = metadata.find("dt", {"class":"toc-pdf"}).find("a")['href']
    
    data.append({"EO_number": eo_number,
                 "title": title, 
                 "URL": url,
                 "signed_date": date_signed,
                 "published_date": date_published,
                 "fr_citation":fr_citation,
                 "fr_doc_number":fr_doc_number,
                 "number_of_pages":number_of_pages,
                 "link_to_pdf":link_to_pdf,
                })

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data)

In [163]:
df

,EO_number,title,URL,signed_date,published_date,fr_citation,fr_doc_number,number_of_pages,link_to_pdf
0,14276,Restoring American Seafood Competitiveness,https://www.federalregister.gov/documents/2025...,"April 17, 2025","April 22, 2025",90 FR 16993,2025-07062,3 pages,https://www.govinfo.gov/content/pkg/FR-2025-04...
1,14275,Restoring Common Sense to Federal Procurement,https://www.federalregister.gov/documents/2025...,"April 15, 2025","April 18, 2025",90 FR 16447,2025-06839,3 pages,https://www.govinfo.gov/content/pkg/FR-2025-04...
2,14274,Restoring Common Sense to Federal Office Space...,https://www.federalregister.gov/documents/2025...,"April 15, 2025","April 18, 2025",90 FR 16445,2025-06838,2 pages,https://www.govinfo.gov/content/pkg/FR-2025-04...
3,14273,Lowering Drug Prices by Once Again Putting Ame...,https://www.federalregister.gov/documents/2025...,"April 15, 2025","April 18, 2025",90 FR 16441,2025-06837,4 pages,https://www.govinfo.gov/content/pkg/FR-2025-04...
4,14272,Ensuring National Security and Economic Resili...,https://www.federalregister.gov/documents/2025...,"April 15, 2025","April 18, 2025",90 FR 16437,2025-06836,4 pages,https://www.govinfo.gov/content/pkg/FR-2025-04...
...,...,...,...,...,...,...,...,...,...
125,14151,Ending Radical and Wasteful Government DEI Pro...,https://www.federalregister.gov/documents/2025...,"January 20, 2025","January 29, 2025",90 FR 8339,2025-01953,3 pages,https://www.govinfo.gov/content/pkg/FR-2025-01...
126,14150,America First Policy Directive to the Secretar...,https://www.federalregister.gov/documents/2025...,"January 20, 2025","January 29, 2025",90 FR 8337,2025-01952,1 page,https://www.govinfo.gov/content/pkg/FR-2025-01...
127,14149,Restoring Freedom of Speech and Ending Federal...,https://www.federalregister.gov/documents/2025...,"January 20, 2025","January 28, 2025",90 FR 8243,2025-01902,2 pages,https://www.govinfo.gov/content/pkg/FR-2025-01...
128,14148,Initial Rescissions of Harmful Executive Order...,https://www.federalregister.gov/documents/2025...,"January 20, 2025","January 28, 2025",90 FR 8237,2025-01901,5 pages,https://www.govinfo.gov/content/pkg/FR-2025-01...


In [156]:
upload_to_google_sheets(SPREADSHEET_URL, "EO_list", df.sort_values(by='EO_number', ascending = True))

In [170]:
#count the EOs by date
df_by_date = df.groupby(by='signed_date')['title'].count().reset_index() 

In [171]:
#rename columns
df_by_date.columns=(['signed_date','number_EOs'])

In [172]:
#group titles by date (for tooltip)
titles_by_date = df.groupby(by='signed_date')['title'].apply(lambda x: '<br><br>'.join(x)).reset_index()

In [173]:
#merge both together
df_by_date=df_by_date.merge(titles_by_date, on ='signed_date')

In [174]:
#adding proxy for charting
df_by_date['proxy']=1

In [175]:
df_by_date.head()

,signed_date,number_EOs,title,proxy
0,"April 15, 2025",5,Restoring Common Sense to Federal Procurement<...,1
1,"April 17, 2025",1,Restoring American Seafood Competitiveness,1
2,"April 2, 2025",2,Regulating Imports With a Reciprocal Tariff To...,1
3,"April 4, 2025",1,Extending the TikTok Enforcement Delay,1
4,"April 8, 2025",4,Strengthening the Reliability and Security of ...,1


In [176]:
upload_to_google_sheets(SPREADSHEET_URL, "EO_grouped_by_date", df_by_date)

## Topic clustering

In [195]:
EO_topic_clustered_url = "https://docs.google.com/spreadsheets/d/1KrxeJ5sPvirGUhL9i0E4QmwiMQg5O8odtTOBXdEGuLg/export?format=csv&gid=958986611"
df_topics= pd.read_csv(EO_topic_clustered_url, usecols = ['EO_number','title','classification Studio Washington'])
df_topics.head()

,EO_number,title,classification Studio Washington
0,14147,Ending the Weaponization of the Federal Govern...,Government
1,14148,Initial Rescissions of Harmful Executive Order...,"Gender/Diversity/Equity/Inclusion,Government"
2,14149,Restoring Freedom of Speech and Ending Federal...,"Freedom of expression,Social Media and technology"
3,14150,America First Policy Directive to the Secretar...,Foreign Policy and Security
4,14151,Ending Radical and Wasteful Government DEI Pro...,"Gender/Diversity/Equity/Inclusion,Government,H..."


In [196]:
# split on comma
df_topics['topic_list'] = df_topics['classification Studio Washington'].str.split(",")

In [197]:
df_topics['topic_list']

0                                           [Government]
1        [Gender/Diversity/Equity/Inclusion, Government]
2      [Freedom of expression, Social Media and techn...
3                          [Foreign Policy and Security]
4      [Gender/Diversity/Equity/Inclusion, Government...
                             ...                        
125    [Economy and resources, Trade, Foreign Policy ...
126                         [Health/Healthcare/Abortion]
127                                         [Government]
128           [Home Affairs/Domestic policy, Government]
129                       [Economy and resources, Trade]
Name: topic_list, Length: 130, dtype: object

In [198]:
single_mentions = []

for i, row in df_topics.iterrows(): #loop over df
    for element in row['topic_list']: 
        single_mentions.append(element) #append each list item to single mentions

In [199]:
counts = Counter(single_mentions) # .value_counts() for lists

In [200]:
counts

Counter({'Government': 34,
         'Gender/Diversity/Equity/Inclusion': 19,
         'Freedom of expression': 5,
         'Social Media and technology': 6,
         'Foreign Policy and Security': 43,
         'Home Affairs/Domestic policy': 24,
         'Environment/Energy/Climate': 11,
         'Economy and resources': 27,
         'Trade': 24,
         'Health/Healthcare/Abortion': 17,
         'Immigration and citizenship': 11,
         'Religion and antisemitism': 4,
         'Military': 8,
         'Other': 3,
         'Education': 8,
         'Tariffs': 15})

In [201]:
topics_df = pd.Series(counts).sort_values(ascending=False).reset_index()
topics_df.columns =(['topic_cluster','occurrence'])
topics_df

,topic_cluster,occurrence
0,Foreign Policy and Security,43
1,Government,34
2,Economy and resources,27
3,Home Affairs/Domestic policy,24
4,Trade,24
5,Gender/Diversity/Equity/Inclusion,19
6,Health/Healthcare/Abortion,17
7,Tariffs,15
8,Environment/Energy/Climate,11
9,Immigration and citizenship,11


In [191]:
upload_to_google_sheets(SPREADSHEET_URL, "EO_clustered", topics_df)